In [80]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt


In [81]:
df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df.index = pd.to_datetime(df.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]
display(df.head(1), df.tail(1))
df.shape


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2010-06-01 00:00:00+00:00,-0.195535,-0.431203,-0.701336,-0.140981,-0.470019,-0.252915,-0.504554,-0.271715,0.354467,0.168679,...,-0.372762,0.185407,-0.157289,-0.047979,-0.305857,-0.55999,-0.335956,0.293033,0.037194,-0.217444


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,-0.05858,-0.048583,0.019215,0.010852,-0.092832,0.009434,0.079595,0.068199,0.036857,-0.023727,...,0.065983,0.070777,-0.053466,-0.080965,-0.077221,-0.004721,-0.121889,0.041164,0.052337,0.119222


(74593, 28)

In [82]:
SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]


In [83]:
def mean_prediction(df, rate=24):
    data = pd.DataFrame([])
    for sym in SYMBOLS:
        data[f'{sym}_tend'] = df[sym].rolling(rate).mean()
    data.index = df.index
    return data
        


In [84]:
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}_pred_vol']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    return data

In [85]:
one = random_prediction(df)
two = mean_prediction(df)


In [86]:
def condition(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0

for sym in two.columns:
    two[sym] = two[sym].map( lambda x: condition(x))
    
# two.reset_index(inplace=True, drop=True)

display(one.shape, two.shape)


(74593, 28)

(74593, 28)

In [87]:
df = two.join(one)

In [88]:
df

,AUD_CAD_tend,AUD_CHF_tend,AUD_JPY_tend,AUD_NZD_tend,AUD_USD_tend,CAD_CHF_tend,CAD_JPY_tend,CHF_JPY_tend,EUR_AUD_tend,EUR_CAD_tend,...,GBP_JPY_pred_vol,GBP_NZD_pred_vol,GBP_USD_pred_vol,NZD_CAD_pred_vol,NZD_CHF_pred_vol,NZD_JPY_pred_vol,NZD_USD_pred_vol,USD_CAD_pred_vol,USD_CHF_pred_vol,USD_JPY_pred_vol
2010-06-01 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,-0.623756,-0.286406,0.350958,0.060577,-0.180396,-0.707350,0.497163,0.364045,-0.513732,0.284779
2010-06-01 01:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,-0.294181,-0.653162,-0.263150,0.541617,-0.485018,-0.251196,0.232082,0.224609,-0.432349,-0.156178
2010-06-01 02:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0.366367,0.268345,-0.693711,0.072593,-0.446731,0.220182,0.583258,-0.648534,-0.283612,-0.738967
2010-06-01 03:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0.371888,0.733841,-0.242406,-0.474669,0.699158,-0.303842,0.105916,0.555333,0.775765,-0.289175
2010-06-01 04:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,-0.732114,-0.414800,-0.086271,0.672017,-0.088268,0.401962,0.279480,-0.653712,0.125513,0.145648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 20:00:00+00:00,1,1,-1,1,1,1,-1,-1,1,1,...,-0.568137,-0.218896,-0.578116,-0.163016,0.570895,0.336795,-0.575962,0.280305,0.110032,-0.247544
2022-06-30 21:00:00+00:00,1,1,-1,-1,1,1,-1,-1,-1,1,...,-0.077700,0.365082,0.314937,0.430518,-0.461590,0.429644,0.526846,0.363992,0.569780,0.456330
2022-06-30 22:00:00+00:00,1,1,-1,1,1,1,-1,-1,1,1,...,-0.461440,-0.561575,-0.454016,0.206202,0.260880,-0.329573,-0.172192,0.684745,-0.339044,0.699053
2022-06-30 23:00:00+00:00,1,1,-1,-1,1,1,-1,-1,-1,1,...,0.708047,-0.222021,-0.711155,0.646782,0.451868,-0.771593,0.165970,-0.027108,-0.791173,0.104811


In [96]:
pred_last = one.iloc[-120:]
pred = df.iloc[-1:]
display(pred_last.tail(1), pred)


,AUD_CAD_pred_vol,AUD_CHF_pred_vol,AUD_JPY_pred_vol,AUD_NZD_pred_vol,AUD_USD_pred_vol,CAD_CHF_pred_vol,CAD_JPY_pred_vol,CHF_JPY_pred_vol,EUR_AUD_pred_vol,EUR_CAD_pred_vol,...,GBP_JPY_pred_vol,GBP_NZD_pred_vol,GBP_USD_pred_vol,NZD_CAD_pred_vol,NZD_CHF_pred_vol,NZD_JPY_pred_vol,NZD_USD_pred_vol,USD_CAD_pred_vol,USD_CHF_pred_vol,USD_JPY_pred_vol
2022-07-01 00:00:00+00:00,0.294364,-0.480493,-0.285773,-0.63767,0.710066,0.728389,-0.008508,-0.130343,0.696997,-0.27198,...,-0.637695,0.598068,-0.578225,-0.119121,-0.588357,-0.369058,0.636828,0.337211,0.635826,-0.415507


,AUD_CAD_tend,AUD_CHF_tend,AUD_JPY_tend,AUD_NZD_tend,AUD_USD_tend,CAD_CHF_tend,CAD_JPY_tend,CHF_JPY_tend,EUR_AUD_tend,EUR_CAD_tend,...,GBP_JPY_pred_vol,GBP_NZD_pred_vol,GBP_USD_pred_vol,NZD_CAD_pred_vol,NZD_CHF_pred_vol,NZD_JPY_pred_vol,NZD_USD_pred_vol,USD_CAD_pred_vol,USD_CHF_pred_vol,USD_JPY_pred_vol
2022-07-01 00:00:00+00:00,1,1,-1,-1,1,1,-1,-1,-1,1,...,-0.637695,0.598068,-0.578225,-0.119121,-0.588357,-0.369058,0.636828,0.337211,0.635826,-0.415507


In [100]:
def pairs_correlation(df, rate=120):
    data = pd.DataFrame([])
    for pair in range(len(df.columns)):
        for second_pair in range(pair, len(df.columns)):
            data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
    data.index = df.index
    return data

cor = pairs_correlation(pred_last)
cor = cor.iloc[-1:]





# data = pd.DataFrame([])
# for sym in SYMBOLS:

    
    
    

/tmp/ipykernel_12594/27620736.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
/tmp/ipykernel_12594/27620736.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
/tmp/ipykernel_12594/27620736.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance

In [105]:
cor

,0_0_corr,0_1_corr,0_2_corr,0_3_corr,0_4_corr,0_5_corr,0_6_corr,0_7_corr,0_8_corr,0_9_corr,...,24_24_corr,24_25_corr,24_26_corr,24_27_corr,25_25_corr,25_26_corr,25_27_corr,26_26_corr,26_27_corr,27_27_corr
2022-07-01 00:00:00+00:00,1.0,-0.001526,0.015151,0.052727,0.027977,-0.013406,0.021789,-0.085979,-0.057169,0.012399,...,1.0,0.073632,0.016341,-0.069037,1.0,0.04596,-0.058269,1.0,0.055384,1.0
